## RAG Day 3

### Expert Question Answerer for InsureLLM

LangChain 1.0 implementation of a RAG pipeline.

Using the VectorStore we created last time (with HuggingFace `all-MiniLM-L6-v2`)

In [3]:
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI

from langchain_chroma import Chroma
from langchain_core.messages import SystemMessage, HumanMessage
from langchain_huggingface import HuggingFaceEmbeddings
import gradio as gr

In [4]:
MODEL = "gpt-4.1-nano"
DB_NAME = "vector_db"
load_dotenv(override=True)

True

### Connect to Chroma; use Hugging Face all-MiniLM-L6-v2

In [5]:
embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
vectorstore = Chroma(persist_directory=DB_NAME, embedding_function=embeddings)

### Set up the 2 key LangChain objects: retriever and llm

#### A sidebar on "temperature":
- Controls how diverse the output is
- A temperature of 0 means that the output should be predictable
- Higher temperature for more variety in answers

Some people describe temperature as being like 'creativity' but that's not quite right
- It actually controls which tokens get selected during inference
- temperature=0 means: always select the token with highest probability
- temperature=1 usually means: a token with 10% probability should be picked 10% of the time

Note: a temperature of 0 doesn't mean outputs will always be reproducible. You also need to set a random seed. We will do that in weeks 6-8. (Even then, it's not always reproducible.)

Note 2: if you want creativity, use the System Prompt!

In [7]:
retriever = vectorstore.as_retriever()
llm = ChatOpenAI(temperature=0, model_name=MODEL)

### These LangChain objects implement the method `invoke()`

In [8]:
retriever.invoke("Who is Avery?")

[Document(id='3ad085a4-ff01-4166-ae6d-320b9f83da7d', metadata={'doc_type': 'employees', 'source': 'knowledge-base/employees/Avery Lancaster.md'}, page_content="## Other HR Notes\n- **Professional Development**: Avery has actively participated in leadership training programs and industry conferences, representing Insurellm and fostering partnerships.  \n- **Diversity & Inclusion Initiatives**: Avery has championed a commitment to diversity in hiring practices, seeing visible improvements in team representation since 2021.  \n- **Work-Life Balance**: Feedback revealed concerns regarding work-life balance, which Avery has approached by implementing flexible working conditions and ensuring regular check-ins with the team.\n- **Community Engagement**: Avery led community outreach efforts, focusing on financial literacy programs, particularly aimed at underserved populations, improving Insurellm's corporate social responsibility image.  \n\nAvery Lancaster has demonstrated resilience and ada

In [9]:
llm.invoke("Who is Avery?")

AIMessage(content="Avery is a given name that can be used for both males and females. It can also be a surname. Without additional context, it's difficult to determine which specific Avery you're referring to. If you can provide more details—such as a full name, profession, or context—I’d be happy to help you with more specific information.", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 67, 'prompt_tokens': 11, 'total_tokens': 78, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4.1-nano-2025-04-14', 'system_fingerprint': 'fp_7f8eb7d1f9', 'id': 'chatcmpl-CtU1HcHjlljkTHUX5ptFFpmsRYtXV', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--8faedadc-05ab-41b8-8c33-2377564a4dc2-0', usage_metadata={'input_tokens':

## Time to put this together!

In [ ]:
SYSTEM_PROMPT_TEMPLATE = """
You are a knowledgeable, friendly assistant representing the company Insurellm.
You are chatting with a user about Insurellm.
If relevant, use the given context to answer any question.
If you don't know the answer, say so.
Please answer in Korean.
Context:
{context}
"""

In [15]:
def answer_question(question: str, history):
    docs = retriever.invoke(question)
    context = "\n\n".join(doc.page_content for doc in docs)
    system_prompt = SYSTEM_PROMPT_TEMPLATE.format(context=context)
    response = llm.invoke([SystemMessage(content=system_prompt), HumanMessage(content=question)])
    return response.content

In [16]:
answer_question("Who is Averi Lancaster?", [])

'Avery Lancaster는 1985년 3월 15일에 태어난 인물로, 현재 샌프란시스코에 거주하며 Insurellm의 공동 창립자이자 CEO입니다. 2015년부터 Insurellm을 공동 창업하여 회사가 보험 기술 분야의 선두주자로 성장하는 데 중요한 역할을 했습니다. 이전에는 Innovate Insurance Solutions에서 선임 제품 관리자였으며, 그곳에서 혁신적인 보험 상품을 개발하였습니다. 2021년 1월부터는 인슈어런스 데이터 엔지니어로 활동하며, 데이터 전략과 멘토링에 기여하고 있습니다.'

## What could possibly come next? 😂

In [17]:
gr.ChatInterface(answer_question).launch()

/Users/imseungmin/Desktop/work/udemy-llm/llm_engineering/.venv/lib/python3.12/site-packages/gradio/chat_interface.py:347: UserWarning: The 'tuples' format for chatbot messages is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  self.chatbot = Chatbot(


* Running on local URL:  http://127.0.0.1:7861
* To create a public link, set `share=True` in `launch()`.


## Admit it - you thought RAG would be more complicated than that!!